In [1]:
import glob
import os
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models 
import torch.optim as optim 
from PIL import Image

In [2]:
!pip install scikit-image

In [3]:
img_dir = os.path.join('./Dataset/face_images')
aug_dir = os.path.join('./Dataset/augment_images')
lab_dir = os.path.join('./Dataset/lab_images')
l_dir = os.path.join('./Dataset/l_images')
a_dir = os.path.join('./Dataset/a_images')
b_dir = os.path.join('./Dataset/b_images')

In [4]:
a_values = []
image_list = glob.glob(os.path.join(a_dir,"*.jpg"))
for i in image_list:
        image = plt.imread(i)
        a_values = np.append(a_values,image.mean())

In [5]:
b_values = []
image_list = glob.glob(os.path.join(b_dir,"*.jpg"))
image_list.sort()
for i in image_list:
        image = plt.imread(i)
        b_values = np.append(b_values,image.mean())

In [6]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

a_values = NormalizeData(a_values)
b_values = NormalizeData(b_values)

In [7]:
def merge(list1, list2):
      
    merged_list = [[list1[i], list2[i]] for i in range(0, len(list1))]
    return merged_list

In [8]:
y_labels = merge(a_values,b_values )

In [9]:
import pandas as pd
input_image_list = glob.glob(os.path.join(l_dir,"*.jpg"))
input_image_list.sort()


In [10]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev) 

## Mean chrominance

In [11]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        
        self.cnn = nn.Sequential(nn.Conv2d(1,3,kernel_size=2, padding=0,stride=2),
                                 nn.ReLU(),
                                        nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                        nn.ReLU(),
                                        nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                        nn.ReLU(),
                                        nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                        nn.ReLU(),
                                        nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                        nn.ReLU(),
                                        nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                        nn.ReLU(),
                                        nn.Conv2d(3,3,kernel_size=2, padding=0,stride=2),
                                        nn.ReLU())
        self.regressor = nn.Linear(3,2)                               
                                 
    def forward(self, input):
        output = self.cnn(input)
        output = output.flatten()
        mean_chrominance = self.regressor(output)
        return  mean_chrominance

In [12]:
# Model
model = CNN()
model.to(device)

CNN(
  (cnn): Sequential(
    (0): Conv2d(1, 3, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (5): ReLU()
    (6): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (7): ReLU()
    (8): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (9): ReLU()
    (10): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (11): ReLU()
    (12): Conv2d(3, 3, kernel_size=(2, 2), stride=(2, 2))
    (13): ReLU()
  )
  (regressor): Linear(in_features=3, out_features=2, bias=True)
)

In [13]:
# Loss and optimizer
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# Train Network
import torchvision.transforms as transforms
trans = transforms.Compose([transforms.ToTensor()])

predictions = []
for epoch in range(10):
    losses = []
    for index in range(len(input_image_list)):
        img_path =  input_image_list[index]
        image = plt.imread(img_path)
        mean, std = image.mean(), image.std()
        image = trans(image)
        y_label = torch.Tensor(y_labels[index]) 
        data = image.to(device=device)
        targets = y_label.to(device=device)
        
        # forward
        pred = model(data)
        predictions.append(pred)
        loss_val = loss(pred, targets)
        
        losses.append(loss_val.item())
        
        optimizer.zero_grad()
        loss_val.backward()

        # gradient descent or adam step
        optimizer.step()
    
    print(np.mean(losses))

0.013456131064227507
0.012503337131130889
0.01248067472119268
0.012480676998275594
0.012480676998275594
0.012480676998275594
0.012480676998275594
0.012480676998275594
0.012480676998275594
0.012480676998275594


In [15]:
print(losses[3])
print(predictions[3])
print(a_values[3])
print(b_values[3])

0.0005302971694618464
tensor([-0.0477,  0.4015], grad_fn=<AddBackward0>)
0.5981083676626149
0.7988074740363509
